In [8]:
pip install streamlit torch transformers datasets peft scikit-learn plotly

In [9]:
%%writefile climate_app.py
import os
import csv
import time
import logging
import tempfile
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import pandas as pd
import streamlit as st
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import plotly.express as px

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Config:
    """Configuration centralisée de l'application"""
    MODEL_NAME = "t5-small"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    TORCH_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

    # Paramètres optimisés pour l'échantillonnage
    DEFAULT_PARAMS = {
        "sample_sizes": {
            "test_rapide": 30000,    # 10K par classe - test en 5 min
            "validation": 75000,     # 25K par classe - validation en 15 min
            "production": 150000,    # 50K par classe - modèle final en 30 min
            "maximum": 300000        # 100K par classe - si nécessaire
        },
        "chunk_size": 20000,
        "test_size": 0.2,
        "val_size": 0.125,
        "max_input_length": 256,
        "max_target_length": 16,
        "train_batch_size": 8,
        "eval_batch_size": 8,
        "num_epochs": 3,
        "learning_rate": 5e-4,
        "lora_r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.05,
        "eval_steps": 50,
        "logging_steps": 25
    }

    LABEL_MAPPING = {"negative": 0, "neutral": 1, "positive": 2}
    LABEL_NAMES = ["negative", "neutral", "positive"]

class SmartSampler:
    """Échantillonneur intelligent pour gros datasets"""

    def __init__(self, config: Config):
        self.config = config

    def estimate_dataset_size(self, file_path: str) -> Dict[str, Any]:
        """Estime la taille et les caractéristiques du dataset"""
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)

        # Estimation du nombre de lignes basée sur la taille du fichier
        # Règle empirique : ~1KB par ligne pour du texte Reddit
        estimated_lines = int(file_size_mb * 1000)

        return {
            "file_size_mb": file_size_mb,
            "estimated_lines": estimated_lines,
            "processing_time_estimate": self._estimate_processing_time(estimated_lines)
        }

    def _estimate_processing_time(self, lines: int) -> Dict[str, str]:
        """Estime les temps de traitement selon la taille"""
        times = {}
        for size_name, sample_size in self.config.DEFAULT_PARAMS["sample_sizes"].items():
            if sample_size >= lines:
                times[size_name] = f"{int(lines / 5000)} min (dataset complet)"
            else:
                times[size_name] = f"{int(sample_size / 5000)} min"
        return times

    def create_stratified_sample(self, df: pd.DataFrame, target_size: int,
                                progress_callback=None) -> pd.DataFrame:
        """Crée un échantillon stratifié intelligent"""

        if progress_callback:
            progress_callback("🔍 Analyse de la distribution des classes...")

        # Analyse de la distribution
        class_counts = df['label'].value_counts().sort_index()
        total_samples = len(df)

        st.info(f"📊 Distribution originale: {dict(class_counts)}")

        # Calcul des tailles par classe pour l'équilibrage
        samples_per_class = target_size // 3  # 3 classes

        if progress_callback:
            progress_callback(f"🎯 Objectif: {samples_per_class:,} échantillons par classe")

        balanced_samples = []

        for class_label in [0, 1, 2]:  # negative, neutral, positive
            class_data = df[df['label'] == class_label]
            available_samples = len(class_data)

            if available_samples == 0:
                st.warning(f"⚠️ Aucun échantillon trouvé pour la classe {self.config.LABEL_NAMES[class_label]}")
                continue

            # Prendre le minimum entre ce qui est disponible et ce qui est demandé
            n_samples = min(samples_per_class, available_samples)

            if progress_callback:
                progress_callback(f"📝 Échantillonnage classe {self.config.LABEL_NAMES[class_label]}: {n_samples:,} échantillons")

            # Échantillonnage aléatoire stratifié
            if n_samples < available_samples:
                sampled_class = class_data.sample(n=n_samples, random_state=42)
            else:
                sampled_class = class_data

            balanced_samples.append(sampled_class)

        # Combinaison et mélange final
        if progress_callback:
            progress_callback("🔄 Combinaison des échantillons...")

        final_sample = pd.concat(balanced_samples, ignore_index=True)
        final_sample = final_sample.sample(frac=1, random_state=42).reset_index(drop=True)

        # Statistiques finales
        final_class_counts = final_sample['label'].value_counts().sort_index()
        st.success(f"✅ Échantillon créé: {dict(final_class_counts)} (Total: {len(final_sample):,})")

        return final_sample

class OptimizedDataProcessor:
    """Processeur de données optimisé pour l'échantillonnage"""

    def __init__(self, config: Config):
        self.config = config
        self.sampler = SmartSampler(config)

    def load_and_sample_data(self, file_path: str, target_sample_size: int) -> Optional[pd.DataFrame]:
        """Charge et échantillonne les données de manière optimisée"""

        try:
            # Estimation initiale
            file_info = self.sampler.estimate_dataset_size(file_path)
            st.info(f"📁 Fichier: {file_info['file_size_mb']:.1f} MB (~{file_info['estimated_lines']:,} lignes estimées)")

            # Interface de progression
            progress_bar = st.progress(0)
            status_text = st.empty()

            def update_progress(message):
                status_text.text(message)

            # Stratégie de chargement basée sur la taille
            if file_info["file_size_mb"] > 500:  # > 500MB
                return self._load_large_file_with_sampling(
                    file_path, target_sample_size, progress_bar, update_progress
                )
            else:
                return self._load_and_sample_standard(
                    file_path, target_sample_size, progress_bar, update_progress
                )

        except Exception as e:
            st.error(f"❌ Erreur lors du traitement: {str(e)}")
            logger.error(f"Erreur traitement données: {e}")
            return None

    def _load_and_sample_standard(self, file_path: str, target_size: int,
                                 progress_bar, update_progress) -> Optional[pd.DataFrame]:
        """Charge un fichier standard et l'échantillonne"""

        update_progress("📖 Lecture du fichier...")
        progress_bar.progress(0.2)

        # Tentative de chargement avec différents encodages
        df = None
        for encoding in ['utf-8', 'latin-1', 'cp1252']:
            try:
                df = pd.read_csv(
                    file_path,
                    encoding=encoding,
                    on_bad_lines='skip',
                    engine='python',
                    quoting=csv.QUOTE_MINIMAL
                )
                logger.info(f"✅ Fichier chargé avec encoding {encoding}")
                break
            except UnicodeDecodeError:
                continue

        if df is None:
            st.error("❌ Impossible de décoder le fichier CSV")
            return None

        progress_bar.progress(0.4)
        update_progress("🧹 Validation et nettoyage...")

        # Validation et nettoyage
        cleaned_df = self._validate_and_clean_data(df)
        if cleaned_df.empty:
            return None

        progress_bar.progress(0.6)

        # Échantillonnage intelligent
        sampled_df = self.sampler.create_stratified_sample(
            cleaned_df, target_size, update_progress
        )

        progress_bar.progress(1.0)
        update_progress(f"✅ Traitement terminé: {len(sampled_df):,} échantillons")

        return sampled_df

    def _load_large_file_with_sampling(self, file_path: str, target_size: int,
                                      progress_bar, update_progress) -> Optional[pd.DataFrame]:
        """Charge un gros fichier avec échantillonnage par chunks"""

        update_progress("🔍 Analyse du gros fichier...")

        # Première passe : estimation et échantillonnage des chunks
        chunk_size = self.config.DEFAULT_PARAMS["chunk_size"]
        sampled_chunks = []
        total_processed = 0

        # Calcul du ratio d'échantillonnage approximatif
        file_info = self.sampler.estimate_dataset_size(file_path)
        if file_info["estimated_lines"] > target_size:
            chunk_sample_ratio = target_size / file_info["estimated_lines"] * 2  # x2 pour avoir de la marge
        else:
            chunk_sample_ratio = 1.0

        try:
            # Lecture par chunks avec échantillonnage
            for encoding in ['utf-8', 'latin-1', 'cp1252']:
                try:
                    chunk_reader = pd.read_csv(
                        file_path,
                        encoding=encoding,
                        chunksize=chunk_size,
                        on_bad_lines='skip',
                        engine='python',
                        quoting=csv.QUOTE_MINIMAL
                    )

                    for i, chunk in enumerate(chunk_reader):
                        # Validation du premier chunk
                        if i == 0:
                            if not self._validate_columns(chunk):
                                return None

                        # Nettoyage du chunk
                        cleaned_chunk = self._clean_chunk(chunk)
                        if len(cleaned_chunk) == 0:
                            continue

                        # Échantillonnage du chunk si nécessaire
                        if chunk_sample_ratio < 1.0:
                            n_samples = max(1, int(len(cleaned_chunk) * chunk_sample_ratio))
                            cleaned_chunk = cleaned_chunk.sample(n=n_samples, random_state=42)

                        sampled_chunks.append(cleaned_chunk)
                        total_processed += len(chunk)

                        # Mise à jour de la progression
                        progress = min(0.4 + (i * 0.4 / 100), 0.8)  # 40-80% pour le chargement
                        progress_bar.progress(progress)
                        update_progress(f"📊 Chunks traités: {i+1} - Lignes: {total_processed:,}")

                        # Arrêt si on a assez de données
                        total_samples = sum(len(chunk) for chunk in sampled_chunks)
                        if total_samples >= target_size * 3:  # x3 pour avoir de la marge avant l'équilibrage
                            break

                    break  # Succès avec cet encoding

                except UnicodeDecodeError:
                    continue

            if not sampled_chunks:
                st.error("❌ Aucune donnée valide trouvée")
                return None

            # Combinaison des chunks
            progress_bar.progress(0.85)
            update_progress("🔄 Assemblage des données...")

            combined_df = pd.concat(sampled_chunks, ignore_index=True)

            # Échantillonnage final stratifié
            progress_bar.progress(0.9)
            final_sample = self.sampler.create_stratified_sample(
                combined_df, target_size, update_progress
            )

            progress_bar.progress(1.0)
            update_progress(f"✅ Gros fichier traité: {len(final_sample):,} échantillons finaux")

            return final_sample

        except Exception as e:
            st.error(f"❌ Erreur lors du traitement du gros fichier: {str(e)}")
            return None

    def _validate_columns(self, df: pd.DataFrame) -> bool:
        """Valide la présence des colonnes requises"""
        required_cols = ["comment_sentiment", "post_title", "self_text"]
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            st.error(f"❌ Colonnes manquantes: {missing_cols}")
            return False
        return True

    def _validate_and_clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Valide et nettoie un DataFrame complet"""
        if not self._validate_columns(df):
            return pd.DataFrame()
        return self._clean_data(df)

    def _clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Nettoie les données"""
        initial_count = len(df)

        # Filtrage des labels valides
        valid_labels = set(self.config.LABEL_MAPPING.keys())
        df = df[df["comment_sentiment"].isin(valid_labels)]

        # Suppression des valeurs manquantes
        df = df.dropna(subset=["comment_sentiment", "post_title", "self_text"])

        # Création des labels numériques
        df["label"] = df["comment_sentiment"].map(self.config.LABEL_MAPPING).astype(int)

        # Création du texte combiné
        df["text"] = (
            df["post_title"].fillna("") + " " + df["self_text"].fillna("")
        ).str.strip()

        # Filtrage des textes vides
        df = df[df["text"].str.len() > 0]

        final_count = len(df)
        if initial_count > 0:
            logger.info(f"Données nettoyées: {initial_count} → {final_count} échantillons ({final_count/initial_count*100:.1f}% conservés)")

        return df[["text", "label", "comment_sentiment"]]

    def _clean_chunk(self, chunk: pd.DataFrame) -> pd.DataFrame:
        """Nettoie un chunk de données"""
        return self._clean_data(chunk)

    def split_data(self, df: pd.DataFrame, test_size: float, val_size: float) -> Dict[str, pd.DataFrame]:
        """Divise les données en train/val/test avec stratification"""
        try:
            train_val, test = train_test_split(
                df, test_size=test_size, random_state=42, stratify=df["label"]
            )

            train, val = train_test_split(
                train_val, test_size=val_size, random_state=42, stratify=train_val["label"]
            )

            return {"train": train, "validation": val, "test": test}

        except Exception as e:
            st.error(f"❌ Erreur lors de la division des données: {str(e)}")
            return {}

class ModelManager:
    """Gestionnaire du modèle et de l'entraînement optimisé"""

    def __init__(self, config: Config):
        self.config = config
        self.tokenizer = None
        self.model = None
        self.trainer = None
        self.training_logs = {"train_loss": [], "val_loss": [], "steps": [], "epoch": []}

    def initialize_model(self, lora_params: Dict[str, Any]) -> bool:
        """Initialise le modèle avec gestion d'erreurs"""
        try:
            with st.spinner("🤖 Chargement du tokenizer..."):
                self.tokenizer = AutoTokenizer.from_pretrained(self.config.MODEL_NAME)
                if self.tokenizer.pad_token is None:
                    self.tokenizer.pad_token = self.tokenizer.eos_token

            with st.spinner("🧠 Chargement du modèle de base..."):
                base_model = AutoModelForSeq2SeqLM.from_pretrained(
                    self.config.MODEL_NAME,
                    torch_dtype=self.config.TORCH_DTYPE,
                    device_map=None
                ).to(self.config.DEVICE)

            with st.spinner("🔧 Configuration LoRA..."):
                lora_config = LoraConfig(
                    task_type=TaskType.SEQ_2_SEQ_LM,
                    r=lora_params["lora_r"],
                    lora_alpha=lora_params["lora_alpha"],
                    target_modules=["q", "v"],
                    lora_dropout=lora_params["lora_dropout"],
                    bias="none"
                )

                self.model = get_peft_model(base_model, lora_config)

            # Affichage des informations du modèle
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            total_params = sum(p.numel() for p in self.model.parameters())

            st.success(f"✅ Modèle initialisé!")
            st.info(f"📊 Paramètres entraînables: {trainable_params:,} / {total_params:,} ({trainable_params/total_params*100:.2f}%)")

            logger.info("✅ Modèle initialisé avec succès")
            return True

        except Exception as e:
            st.error(f"❌ Erreur d'initialisation du modèle: {str(e)}")
            logger.error(f"Erreur initialisation modèle: {e}")
            return False

    def preprocess_data(self, examples: Dict[str, List], max_input_length: int, max_target_length: int):
        """Préprocesse les données pour l'entraînement"""
        inputs = [f"classify sentiment: {text}" for text in examples["text"]]
        targets = [self.config.LABEL_NAMES[label] for label in examples["label"]]

        model_inputs = self.tokenizer(
            inputs,
            max_length=max_input_length,
            truncation=True,
            padding=True,
            return_tensors="pt" if len(inputs) == 1 else None
        )

        labels = self.tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding=True,
            return_tensors="pt" if len(targets) == 1 else None
        )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def compute_metrics(self, eval_pred) -> Dict[str, float]:
        """Calcule les métriques d'évaluation de manière robuste"""
        try:
            predictions = eval_pred.predictions[0]
            labels = eval_pred.label_ids

            # Décodage des prédictions
            decoded_preds = []
            for pred in predictions:
                if isinstance(pred, np.ndarray):
                    pred_ids = np.argmax(pred, axis=-1) if pred.ndim > 1 else pred
                else:
                    pred_ids = pred

                decoded_text = self.tokenizer.decode(pred_ids, skip_special_tokens=True).strip().lower()

                # Mapping robuste des prédictions
                if "negative" in decoded_text:
                    decoded_preds.append(0)
                elif "positive" in decoded_text:
                    decoded_preds.append(2)
                else:
                    decoded_preds.append(1)  # neutral par défaut

            # Décodage des labels
            decoded_labels = []
            for label in labels:
                if hasattr(label, '__iter__') and not isinstance(label, str):
                    label_text = self.tokenizer.decode(label, skip_special_tokens=True).strip().lower()
                    if "negative" in label_text:
                        decoded_labels.append(0)
                    elif "positive" in label_text:
                        decoded_labels.append(2)
                    else:
                        decoded_labels.append(1)
                else:
                    decoded_labels.append(int(label))

            return {
                "accuracy": accuracy_score(decoded_labels, decoded_preds),
                "f1_weighted": f1_score(decoded_labels, decoded_preds, average="weighted"),
                "f1_macro": f1_score(decoded_labels, decoded_preds, average="macro")
            }

        except Exception as e:
            logger.error(f"Erreur calcul métriques: {e}")
            return {"accuracy": 0.0, "f1_weighted": 0.0, "f1_macro": 0.0}

    def setup_trainer(self, datasets: Dict[str, Dataset], training_params: Dict[str, Any]) -> bool:
        """Configure le trainer optimisé"""
        try:
            # Callback pour tracker les losses
            class LossTrackingCallback(TrainerCallback):
                def __init__(self, logs_dict):
                    self.logs = logs_dict

                def on_log(self, args, state, control, logs=None, **kwargs):
                    if logs:
                        if "loss" in logs:
                            self.logs["train_loss"].append(logs["loss"])
                            self.logs["steps"].append(state.global_step)
                            self.logs["epoch"].append(state.epoch)
                        if "eval_loss" in logs:
                            self.logs["val_loss"].append(logs["eval_loss"])

            # Configuration optimisée de l'entraînement
            training_args = TrainingArguments(
                output_dir="./lora_climate_model",
                per_device_train_batch_size=training_params["train_batch_size"],
                per_device_eval_batch_size=training_params["eval_batch_size"],
                num_train_epochs=training_params["num_epochs"],
                learning_rate=training_params["learning_rate"],
                warmup_steps=100,  # Warm-up pour stabiliser l'entraînement
                eval_strategy="steps",
                eval_steps=training_params["eval_steps"],
                logging_steps=training_params["logging_steps"],
                save_strategy="steps",
                save_steps=training_params["eval_steps"],
                load_best_model_at_end=True,
                metric_for_best_model="eval_f1_weighted",
                greater_is_better=True,
                report_to=None,
                dataloader_pin_memory=False,
                remove_unused_columns=True,
                push_to_hub=False,
                fp16=torch.cuda.is_available(),  # Optimisation mémoire si GPU
            )

            self.trainer = Trainer(
                model=self.model,
                args=training_args,
                train_dataset=datasets["train"],
                eval_dataset=datasets["validation"],
                tokenizer=self.tokenizer,
                compute_metrics=self.compute_metrics,
                callbacks=[LossTrackingCallback(self.training_logs)]
            )

            return True

        except Exception as e:
            st.error(f"❌ Erreur configuration trainer: {str(e)}")
            logger.error(f"Erreur setup trainer: {e}")
            return False

def create_streamlit_app():
    """Interface Streamlit optimisée pour l'Option A"""

    st.set_page_config(
        page_title="🌍 Climate Sentiment AI - Option A",
        page_icon="🌍",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("🌍 Climate Sentiment AI - Option A : Échantillonnage Intelligent")
    st.markdown("*Optimisé pour traiter efficacement des datasets de 1,2 Go avec échantillonnage stratifié*")
    st.markdown("---")

    # Initialisation des objets
    config = Config()
    data_processor = OptimizedDataProcessor(config)
    model_manager = ModelManager(config)

    # Sidebar optimisée pour l'échantillonnage
    st.sidebar.header("⚙️ Configuration Option A")

    # Sélection de la stratégie d'échantillonnage
    st.sidebar.subheader("🎯 Stratégie d'échantillonnage")

    sample_strategies = {
        "🚀 Test rapide (30K)": {
            "size": config.DEFAULT_PARAMS["sample_sizes"]["test_rapide"],
            "description": "Validation rapide en 5 min",
            "use_case": "Test de faisabilité"
        },
        "✅ Validation (75K)": {
            "size": config.DEFAULT_PARAMS["sample_sizes"]["validation"],
            "description": "Équilibre temps/qualité en 15 min",
            "use_case": "Développement et test"
        },
        "🎯 Production (150K)": {
            "size": config.DEFAULT_PARAMS["sample_sizes"]["production"],
            "description": "Modèle final en 30 min",
            "use_case": "Modèle de production"
        },
        "🔥 Maximum (300K)": {
            "size": config.DEFAULT_PARAMS["sample_sizes"]["maximum"],
            "description": "Performance maximale en 60 min",
            "use_case": "Si qualité insuffisante"
        }
    }

    selected_strategy = st.sidebar.selectbox(
        "Choisir la stratégie",
        options=list(sample_strategies.keys()),
        index=2,  # Production par défaut
        help="Choisissez selon vos contraintes de temps et qualité"
    )

    strategy_info = sample_strategies[selected_strategy]
    target_sample_size = strategy_info["size"]

    # Affichage des informations de la stratégie
    st.sidebar.info(f"""
    **{selected_strategy}**

    📊 Échantillons: {target_sample_size:,}
    ⏱️ Temps estimé: {strategy_info['description']}
    🎯 Usage: {strategy_info['use_case']}
    """)

    # Upload de fichier
    st.sidebar.subheader("📁 Fichier de données")
    uploaded_file = st.sidebar.file_uploader(
        "Charger fichier CSV",
        type=["csv"],
        help="Fichier avec colonnes: comment_sentiment, post_title, self_text"
    )

    # Option de chemin local pour très gros fichiers
    st.sidebar.markdown("**Pour fichiers > 200MB:**")
    local_file_path = st.sidebar.text_input(
        "Chemin fichier local",
        placeholder="/path/to/large_file.csv",
        help="Contourner la limite Streamlit"
    )
    use_local_file = st.sidebar.button("📂 Utiliser fichier local")

    # Paramètres d'entraînement
    st.sidebar.subheader("🏋️ Paramètres d'entraînement")
    num_epochs = st.sidebar.slider("Époques", 1, 5, config.DEFAULT_PARAMS["num_epochs"])
    learning_rate = st.sidebar.select_slider(
        "Learning rate",
        options=[1e-5, 5e-5, 1e-4, 5e-4, 1e-3],
        value=config.DEFAULT_PARAMS["learning_rate"],
        format_func=lambda x: f"{x:.0e}"
    )
    batch_size = st.sidebar.selectbox("Batch size", [4, 8, 16], index=1)

    # Interface principale
    col1, col2 = st.columns([2, 1])

    with col1:
        st.header("📊 Tableau de bord")

        # Détermination de la source de données
        data_source = None
        tmp_file_path = None

        if uploaded_file is not None:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp_file:
                tmp_file.write(uploaded_file.getvalue())
                tmp_file_path = tmp_file.name
            data_source = tmp_file_path

        elif use_local_file and local_file_path.strip():
            if os.path.exists(local_file_path.strip()):
                data_source = local_file_path.strip()
            else:
                st.error(f"❌ Fichier non trouvé: {local_file_path}")

        # Traitement des données
        if data_source:
            try:
                st.subheader("🎯 Échantillonnage intelligent")

                # Traitement des données avec échantillonnage
                df = data_processor.load_and_sample_data(data_source, target_sample_size)

                if df is not None:
                    # Division des données
                    data_splits = data_processor.split_data(
                        df,
                        config.DEFAULT_PARAMS["test_size"],
                        config.DEFAULT_PARAMS["val_size"]
                    )

                    if data_splits:
                        # Statistiques détaillées
                        st.subheader("📈 Statistiques de l'échantillon")

                        # Métriques principales
                        metrics_col1, metrics_col2, metrics_col3, metrics_col4 = st.columns(4)

                        with metrics_col1:
                            st.metric("📊 Total", f"{len(df):,}")
                        with metrics_col2:
                            st.metric("🏋️ Train", f"{len(data_splits['train']):,}")
                        with metrics_col3:
                            st.metric("✅ Validation", f"{len(data_splits['validation']):,}")
                        with metrics_col4:
                            st.metric("🧪 Test", f"{len(data_splits['test']):,}")

                        # Visualisations
                        viz_col1, viz_col2 = st.columns(2)

                        with viz_col1:
                            # Distribution des sentiments
                            sentiment_counts = df['comment_sentiment'].value_counts()
                            fig_pie = px.pie(
                                values=sentiment_counts.values,
                                names=sentiment_counts.index,
                                title="Distribution des sentiments",
                                color_discrete_map={
                                    'negative': '#ff6b6b',
                                    'neutral': '#ffd93d',
                                    'positive': '#6bcf7f'
                                }
                            )
                            st.plotly_chart(fig_pie, use_container_width=True)

                        with viz_col2:
                            # Distribution par split
                            split_data = []
                            for split_name, split_df in data_splits.items():
                                for sentiment in ['negative', 'neutral', 'positive']:
                                    count = len(split_df[split_df['comment_sentiment'] == sentiment])
                                    split_data.append({
                                        'Split': split_name.capitalize(),
                                        'Sentiment': sentiment,
                                        'Count': count
                                    })

                            split_df_viz = pd.DataFrame(split_data)
                            fig_bar = px.bar(
                                split_df_viz,
                                x='Split',
                                y='Count',
                                color='Sentiment',
                                title="Distribution par split",
                                color_discrete_map={
                                    'negative': '#ff6b6b',
                                    'neutral': '#ffd93d',
                                    'positive': '#6bcf7f'
                                }
                            )
                            st.plotly_chart(fig_bar, use_container_width=True)

                        # Aperçu des données
                        st.subheader("👀 Aperçu des données")
                        sample_data = df.sample(n=min(5, len(df)), random_state=42)

                        for idx, row in sample_data.iterrows():
                            sentiment_color = {
                                'negative': '🔴',
                                'neutral': '🟡',
                                'positive': '🟢'
                            }

                            with st.expander(f"{sentiment_color[row['comment_sentiment']]} {row['comment_sentiment'].upper()} - Échantillon {idx}"):
                                st.write(f"**Texte:** {row['text'][:200]}...")

                        # Stockage dans session state
                        st.session_state["data_splits"] = data_splits
                        st.session_state["data_ready"] = True
                        st.session_state["sample_strategy"] = selected_strategy

                        # Informations sur la stratégie utilisée
                        st.success(f"✅ {selected_strategy} appliquée avec succès!")

            finally:
                # Nettoyage du fichier temporaire
                if tmp_file_path and os.path.exists(tmp_file_path):
                    os.unlink(tmp_file_path)

        elif "data_ready" not in st.session_state:
            st.info("👆 Veuillez charger un fichier CSV pour commencer l'échantillonnage intelligent")

            # Guide d'utilisation
            st.markdown("""
            ### 🎯 Guide d'utilisation Option A

            **1. Choisissez votre stratégie d'échantillonnage:**
            - 🚀 **Test rapide (30K)**: Pour valider rapidement votre pipeline
            - ✅ **Validation (75K)**: Bon équilibre pour le développement
            - 🎯 **Production (150K)**: Modèle final de qualité production
            - 🔥 **Maximum (300K)**: Performance maximale si nécessaire

            **2. Chargez vos données:**
            - Fichiers < 200MB: Upload direct
            - Fichiers > 200MB: Chemin local

            **3. L'algorithme va:**
            - Analyser votre dataset
            - Créer un échantillon équilibré et représentatif
            - Optimiser pour vos contraintes de temps
            """)

    with col2:
        st.header("🚀 Actions")

        # Informations sur le GPU/CPU
        device_info = "🔥 GPU" if config.DEVICE == "cuda" else "💻 CPU"
        st.info(f"**Dispositif:** {device_info}")

        if "data_ready" in st.session_state:
            strategy_used = st.session_state.get("sample_strategy", "Non définie")
            st.success(f"**Stratégie:** {strategy_used}")

        # Bouton d'initialisation du modèle
        if st.button("🤖 Initialiser Modèle", use_container_width=True):
            if "data_ready" in st.session_state:
                lora_params = {
                    "lora_r": config.DEFAULT_PARAMS["lora_r"],
                    "lora_alpha": config.DEFAULT_PARAMS["lora_alpha"],
                    "lora_dropout": config.DEFAULT_PARAMS["lora_dropout"]
                }

                if model_manager.initialize_model(lora_params):
                    st.session_state["model_ready"] = True
            else:
                st.warning("⚠️ Chargez d'abord les données")

        # Bouton d'entraînement
        if st.button("🏋️ Lancer Entraînement", use_container_width=True):
            if "model_ready" in st.session_state and "data_ready" in st.session_state:

                # Estimation du temps d'entraînement
                data_splits = st.session_state["data_splits"]
                train_size = len(data_splits["train"])
                estimated_time = (train_size * num_epochs * batch_size) // 1000  # Estimation approximative

                st.info(f"⏱️ Temps estimé: ~{estimated_time} minutes")

                with st.spinner("🏋️ Entraînement en cours..."):
                    try:
                        # Préparation des datasets
                        datasets = {}

                        for split_name, split_data in data_splits.items():
                            dataset = Dataset.from_pandas(split_data)
                            dataset = dataset.map(
                                lambda x: model_manager.preprocess_data(
                                    x,
                                    config.DEFAULT_PARAMS["max_input_length"],
                                    config.DEFAULT_PARAMS["max_target_length"]
                                ),
                                batched=True,
                                remove_columns=split_data.columns.tolist()
                            )
                            dataset.set_format("torch")
                            datasets[split_name] = dataset

                        # Configuration du trainer
                        training_params = {
                            "train_batch_size": batch_size,
                            "eval_batch_size": batch_size,
                            "num_epochs": num_epochs,
                            "learning_rate": learning_rate,
                            "eval_steps": config.DEFAULT_PARAMS["eval_steps"],
                            "logging_steps": config.DEFAULT_PARAMS["logging_steps"]
                        }

                        if model_manager.setup_trainer(datasets, training_params):
                            # Lancement de l'entraînement
                            start_time = time.time()
                            model_manager.trainer.train()
                            training_time = time.time() - start_time

                            st.session_state["training_complete"] = True
                            st.session_state["training_time"] = training_time

                            st.success(f"✅ Entraînement terminé en {training_time/60:.1f} minutes!")

                    except Exception as e:
                        st.error(f"❌ Erreur pendant l'entraînement: {str(e)}")
                        logger.error(f"Erreur entraînement: {e}")
            else:
                st.warning("⚠️ Initialisez d'abord le modèle")

        # Bouton d'évaluation
        if st.button("📊 Évaluer sur Test", use_container_width=True):
            if "training_complete" in st.session_state:
                with st.spinner("📊 Évaluation en cours..."):
                    try:
                        data_splits = st.session_state["data_splits"]
                        test_dataset = Dataset.from_pandas(data_splits["test"])
                        test_dataset = test_dataset.map(
                            lambda x: model_manager.preprocess_data(
                                x,
                                config.DEFAULT_PARAMS["max_input_length"],
                                config.DEFAULT_PARAMS["max_target_length"]
                            ),
                            batched=True,
                            remove_columns=data_splits["test"].columns.tolist()
                        )
                        test_dataset.set_format("torch")

                        results = model_manager.trainer.evaluate(test_dataset)
                        st.session_state["test_results"] = results

                        # Affichage des résultats avec contexte
                        st.subheader("🎯 Résultats finaux")

                        col1, col2, col3 = st.columns(3)
                        with col1:
                            acc = results.get('eval_accuracy', 0)
                            st.metric("🎯 Accuracy", f"{acc:.3f}", delta=f"{(acc-0.33)*100:+.1f}%" if acc > 0.33 else None)
                        with col2:
                            f1w = results.get('eval_f1_weighted', 0)
                            st.metric("📊 F1 Weighted", f"{f1w:.3f}")
                        with col3:
                            f1m = results.get('eval_f1_macro', 0)
                            st.metric("📈 F1 Macro", f"{f1m:.3f}")

                        # Interprétation des résultats
                        if acc > 0.75:
                            st.success("🎉 Excellents résultats! Modèle prêt pour la production.")
                        elif acc > 0.65:
                            st.info("✅ Bons résultats. Considérez la stratégie 'Maximum' pour améliorer.")
                        else:
                            st.warning("⚠️ Résultats moyens. Essayez avec plus de données ou ajustez les paramètres.")

                    except Exception as e:
                        st.error(f"❌ Erreur pendant l'évaluation: {str(e)}")
                        logger.error(f"Erreur évaluation: {e}")
            else:
                st.warning("⚠️ Terminez d'abord l'entraînement")

        # Informations de performance
        if "training_complete" in st.session_state:
            st.markdown("---")
            st.subheader("⚡ Performance")

            training_time = st.session_state.get("training_time", 0)
            strategy_used = st.session_state.get("sample_strategy", "Non définie")

            st.metric("⏱️ Temps d'entraînement", f"{training_time/60:.1f} min")
            st.info(f"**Stratégie utilisée:** {strategy_used}")

    # Onglets pour les visualisations avancées
    if "training_complete" in st.session_state:
        st.markdown("---")
        tab1, tab2, tab3 = st.tabs(["📈 Courbes d'apprentissage", "🔍 Test interactif", "📋 Rapport détaillé"])

        with tab1:
            if model_manager.training_logs["train_loss"]:
                # Graphique des losses
                fig = go.Figure()

                fig.add_trace(go.Scatter(
                    x=model_manager.training_logs["steps"],
                    y=model_manager.training_logs["train_loss"],
                    mode='lines+markers',
                    name='Train Loss',
                    line=dict(color='blue', width=2)
                ))

                if model_manager.training_logs["val_loss"]:
                    val_steps = model_manager.training_logs["steps"][:len(model_manager.training_logs["val_loss"])]
                    fig.add_trace(go.Scatter(
                        x=val_steps,
                        y=model_manager.training_logs["val_loss"],
                        mode='lines+markers',
                        name='Validation Loss',
                        line=dict(color='red', width=2)
                    ))

                fig.update_layout(
                    title="Évolution des losses pendant l'entraînement",
                    xaxis_title="Steps",
                    yaxis_title="Loss",
                    hovermode='x unified',
                    template="plotly_white"
                )

                st.plotly_chart(fig, use_container_width=True)

                # Analyse de la convergence
                if len(model_manager.training_logs["train_loss"]) > 5:
                    last_losses = model_manager.training_logs["train_loss"][-5:]
                    loss_trend = (last_losses[-1] - last_losses[0]) / last_losses[0] * 100

                    if loss_trend < -1:
                        st.success(f"📈 Modèle en cours d'amélioration (-{abs(loss_trend):.1f}% sur les derniers steps)")
                    elif loss_trend > 1:
                        st.warning(f"📉 Loss en augmentation (+{loss_trend:.1f}% - possible surentraînement)")
                    else:
                        st.info("📊 Loss stabilisée - Convergence atteinte")
            else:
                st.info("Aucune donnée d'entraînement disponible")

        with tab2:
            st.subheader("🔍 Test de prédiction interactif")

            # Exemples prédéfinis
            example_texts = {
                "Négatif": "Climate change is destroying our planet and governments are doing nothing about it!",
                "Neutre": "Scientists published a new study about climate change impacts on weather patterns.",
                "Positif": "Great progress on renewable energy! Solar panels are becoming more efficient and affordable."
            }

            col1, col2 = st.columns([2, 1])

            with col1:
                test_text = st.text_area(
                    "Entrez un texte à classifier:",
                    value=example_texts["Neutre"],
                    height=100
                )

            with col2:
                st.write("**Exemples:**")
                for label, text in example_texts.items():
                    if st.button(f"📝 {label}", key=f"example_{label}"):
                        st.session_state["test_text"] = text
                        st.experimental_rerun()

            if st.session_state.get("test_text"):
                test_text = st.session_state["test_text"]

            if st.button("🎯 Prédire", use_container_width=True) and test_text.strip():
                try:
                    # Préparation de l'input
                    input_text = f"classify sentiment: {test_text}"
                    inputs = model_manager.tokenizer(
                        input_text,
                        return_tensors="pt",
                        max_length=config.DEFAULT_PARAMS["max_input_length"],
                        truncation=True,
                        padding=True
                    ).to(config.DEVICE)

                    # Prédiction avec probabilités
                    with torch.no_grad():
                        outputs = model_manager.model.generate(
                            **inputs,
                            max_length=config.DEFAULT_PARAMS["max_target_length"],
                            num_beams=3,
                            early_stopping=True,
                            return_dict_in_generate=True,
                            output_scores=True
                        )

                    predicted_text = model_manager.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

                    # Affichage du résultat avec style
                    sentiment_styles = {
                        "negative": {"color": "#ff6b6b", "icon": "🔴", "bg": "#ffe6e6"},
                        "neutral": {"color": "#ffd93d", "icon": "🟡", "bg": "#fffacd"},
                        "positive": {"color": "#6bcf7f", "icon": "🟢", "bg": "#e6ffe6"}
                    }

                    predicted_sentiment = predicted_text.lower()
                    if predicted_sentiment in sentiment_styles:
                        style = sentiment_styles[predicted_sentiment]

                        st.markdown(f"""
                        <div style="padding: 20px; border-radius: 10px; background-color: {style['bg']}; border-left: 5px solid {style['color']};">
                            <h3 style="color: {style['color']}; margin: 0;">
                                {style['icon']} Prédiction: {predicted_sentiment.upper()}
                            </h3>
                            <p style="margin: 10px 0 0 0; font-style: italic;">"{test_text}"</p>
                        </div>
                        """, unsafe_allow_html=True)
                    else:
                        st.success(f"**Prédiction:** {predicted_text.upper()}")

                except Exception as e:
                    st.error(f"❌ Erreur de prédiction: {str(e)}")

        with tab3:
            st.subheader("📋 Rapport détaillé de l'entraînement")

            # Résumé de la configuration
            config_summary = {
                "Stratégie d'échantillonnage": st.session_state.get("sample_strategy", "Non définie"),
                "Taille de l'échantillon": f"{len(st.session_state.get('data_splits', {}).get('train', [])):,} (train)",
                "Époques": num_epochs,
                "Learning rate": f"{learning_rate:.0e}",
                "Batch size": batch_size,
                "Dispositif": config.DEVICE.upper(),
                "Temps d'entraînement": f"{st.session_state.get('training_time', 0)/60:.1f} min"
            }

            col1, col2 = st.columns(2)

            with col1:
                st.markdown("**Configuration:**")
                for key, value in config_summary.items():
                    st.write(f"• **{key}:** {value}")

            with col2:
                if "test_results" in st.session_state:
                    results = st.session_state["test_results"]
                    st.markdown("**Métriques finales:**")
                    st.write(f"• **Accuracy:** {results.get('eval_accuracy', 0):.3f}")
                    st.write(f"• **F1 Weighted:** {results.get('eval_f1_weighted', 0):.3f}")
                    st.write(f"• **F1 Macro:** {results.get('eval_f1_macro', 0):.3f}")
                    st.write(f"• **Loss finale:** {results.get('eval_loss', 0):.3f}")

            # Recommandations
            st.markdown("---")
            st.markdown("**🎯 Recommandations pour améliorer les performances:**")

            if "test_results" in st.session_state:
                acc = st.session_state["test_results"].get('eval_accuracy', 0)

                if acc < 0.65:
                    st.markdown("""
                    - 📈 **Augmenter la taille de l'échantillon** (stratégie Maximum)
                    - 🔧 **Ajuster le learning rate** (essayer 1e-4 ou 1e-3)
                    - 📚 **Augmenter le nombre d'époques** (5-7 époques)
                    - 🎯 **Vérifier la qualité des données** (textes trop courts, labels incorrects)
                    """)
                elif acc < 0.75:
                    st.markdown("""
                    - ✅ **Bon modèle!** Considérez la stratégie Maximum pour gagner quelques points
                    - 🔧 **Fine-tuning des hyperparamètres** (LoRA rank, alpha)
                    - 📊 **Analyse des erreurs** sur les prédictions incorrectes
                    """)
                else:
                    st.markdown("""
                    - 🎉 **Excellent modèle!** Prêt pour la production
                    - 💾 **Sauvegarder le modèle** pour utilisation future
                    - 🚀 **Déploiement recommandé**
                    """)

if __name__ == "__main__":
    create_streamlit_app()

Overwriting climate_app.py


In [10]:
get_ipython().system_raw('streamlit run climate_app.py --server.port=8501 --server.address=0.0.0.0 &')

In [11]:
pip install pyngrok

In [12]:
import os, time, subprocess, socket
from pyngrok import ngrok

# 1️⃣ Nettoie tout
subprocess.run(["pkill", "-9", "-f", "streamlit"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "ngrok"],   stderr=subprocess.DEVNULL)
ngrok.kill()
time.sleep(2)

# 2️⃣ Démarre Streamlit en arrière-plan
subprocess.Popen(
    ["streamlit", "run", "climate_app.py", "--server.port=8501", "--server.address=0.0.0.0"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# 3️⃣ Attend que le port 8501 soit vraiment écouté
def wait_for_port(port=8501, timeout=10):
    for _ in range(timeout):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", port)) == 0:
                return True
        time.sleep(1)
    return False

if wait_for_port():
    # 4️⃣ Reconnecte ngrok
    ngrok.set_auth_token("30Nciu2LDo3NzmKva2zibt2sCFL_7Ag5r9kUYyBCha12WSZ3")
    public_url = ngrok.connect(8501)
    print("🔗 Accès public :", public_url)
else:
    print("❌ Streamlit n’a pas démarré sur le port 8501")

🔗 Accès public : NgrokTunnel: "https://8c4f2b343f54.ngrok-free.app" -> "http://localhost:8501"
